<a href="https://colab.research.google.com/github/jaejunchoe/HAIDS-Lab/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import Dataset


def date(f='%Y-%m-%d %H:%M:%S'):
    return time.strftime(f, time.localtime())


def calculate_mse(model, dataloader, device):
    mse, sample_count = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            user_reviews, item_reviews, reviews, ratings, user_ids, item_ids = [x.to(device) for x in batch]
            latent, predict = model(user_reviews, item_reviews, user_ids, item_ids)
            mse += F.mse_loss(predict, ratings, reduction='sum').item()
            sample_count += len(ratings)
    return mse / sample_count  # mse of dataloader



class TransNetsDataset(Dataset):
    # 초기화 매서드(데이터셋 초기화 및 필요 변수 설정)
    def __init__(self, data_path, word_dict, config):
        self.word_dict = word_dict
        self.r_count = config.review_count
        self.r_length = config.review_length
        self.lowest_r_count = config.lowest_review_count
        self.PAD_WORD_idx = word_dict[config.PAD_WORD]

        ## 데이터 읽기
        df = pd.read_csv(data_path, header=None, names=['userID', 'itemID', 'review', 'rating'])

        ## userID와 itemID를 숫자로 변환
        df['userID'] = df['userID'].astype('category').cat.codes
        df['itemID'] = df['itemID'].astype('category').cat.codes

        ## 리뷰 데이터를 ID로 변환
        df['review'] = df['review'].apply(self._review2id)
        self.null_idx = set()

        ## 사용자 및 아이템 리뷰 생성
        user_reviews = self._get_reviews(df)
        item_reviews = self._get_reviews(df, 'itemID', 'userID')

        ## 리뷰, 평점, 사용자 및 아이템 ID를 텐서로 변환
        reviews = [self._adjust_review_list([x], 1, self.r_length) for x in df['review']]
        reviews = torch.LongTensor(reviews).view(-1, self.r_length)
        rating = torch.Tensor(df['rating'].to_list()).view(-1, 1)
        user_ids = torch.LongTensor(df['userID'].to_list()).view(-1, 1)
        item_ids = torch.LongTensor(df['itemID'].to_list()).view(-1, 1)

        self.user_reviews = user_reviews
        self.item_reviews = item_reviews
        self.reviews = reviews
        self.rating = rating
        self.user_ids = user_ids
        self.item_ids = item_ids

    # 인덱싱 매서드(특정 인덱스에 해당되는 샘플 데이터를 반환)
    def __getitem__(self, idx):
        return self.user_reviews[idx], self.item_reviews[idx], self.reviews[idx], self.rating[idx],\
                self.user_ids[idx], self.item_ids[idx]

    # 길이 메서드(데이터셋 전체 샘플 수를 반환)
    def __len__(self):
        return self.rating.shape[0]

    # 사용자와 아이템 리뷰 생성 메서드 (특정 사용자 또는 아이템과 연관된 리뷰를 반환)
    def _get_reviews(self, df, lead='userID', costar='itemID'):
        reviews_by_lead = dict(list(df[[costar, 'review']].groupby(df[lead])))
        lead_reviews = []
        for idx, (lead_id, costar_id) in enumerate(zip(df[lead], df[costar])):

        ## 관련없는 리뷰만 갖도록 필터링
            df_data = reviews_by_lead[lead_id]
            reviews = df_data['review'][df_data[costar] != costar_id].to_list()

        ## 리뷰 정렬 및 변환
            reviews = self._adjust_review_list(reviews, self.r_count, self.r_length)
            lead_reviews.append(reviews)
        return torch.LongTensor(lead_reviews)


    # 리뷰 리스트 조정 메서드 (일정 개수(10개)와 길이(80개)로 조절)
    def _adjust_review_list(self, reviews, r_count, r_length):
        reviews = reviews[:r_count] + [[self.PAD_WORD_idx] * r_length] * (r_count - len(reviews))  # Certain count.
        reviews = [r[:r_length] + [0] * (r_length - len(r)) for r in reviews]  # Certain length of review.
        return reviews

    # 리뷰를 단어 ID로 변환하는 매서드
    def _review2id(self, review):
        if not isinstance(review, str):  # 리뷰가 문자열인지 확인
            return []  # 문자열이 아니면 빈 리스트 반환
        return [self.word_dict.get(word, self.PAD_WORD_idx) for word in review.split()]

